In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from sklearn.metrics import r2_score

In [8]:
import plotly.graph_objects as go
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_forecast_component_plotly
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
credentials = service_account.Credentials.from_service_account_file('tidy-hold-359719-964ba4f0d1d8.json') #credenciales
project_id = 'tidy-hold-359719' #nombre del proyecto
client = bigquery.Client(credentials= credentials,project=project_id)
query="""SELECT * FROM `tidy-hold-359719.comision_latinoamericana.temperatura`"""
temperatura = client.query(query).to_dataframe()
query="""SELECT * FROM `tidy-hold-359719.comision_latinoamericana.datos_ONU`"""
datos_ONU = client.query(query).to_dataframe()
query="""SELECT * FROM `tidy-hold-359719.comision_latinoamericana.energyco2`"""
energyco2 = client.query(query).to_dataframe()

In [3]:
ts1 = temperatura.groupby('Anio').mean().reset_index()

In [190]:
#tabla = energyco2[(energyco2.Pais != 'Estados Unidos')]
#tabla = tabla[(tabla.Pais != 'Canadá')]
tabla = energyco2[(energyco2.Pais != 'Estados Unidos') & (energyco2.Pais != 'Canadá')]
ts2 = tabla.drop(tabla[tabla['Emisiones_de_CO2'] == -1].index).groupby('Anio').sum().reset_index()

In [ ]:
datos_ONU = datos_ONU[(datos_ONU.Pais != 'Estados Unidos') & (datos_ONU.Pais != 'Canadá')]

In [303]:
#tabla = datos_ONU[['Anio','intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI']]
ts3 = tabla.drop(tabla[tabla['intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI']==-1].index).groupby('Anio').mean().reset_index().rename(columns={'intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI':'intensidad'})

In [304]:
datos_ONU['prom'] = (datos_ONU['proporcion_de_la_poblacion_con_acceso_a_elecricidad'] + datos_ONU['proporcion_de_la_poblacion_con_dependencia_primaria_a_energias_limpias'])/2
ts4 = datos_ONU[['Anio','prom']].groupby('Anio').mean().reset_index()

In [305]:
#tabla = datos_ONU[(datos_ONU.Pais != 'Estados Unidos')]
#tabla = tabla[(tabla.Pais != 'Canadá')]
ts5 = tabla[['Anio', 'proporcion_de_energias_renovables_del_total_consumido']].groupby('Anio').mean().reset_index().rename(columns={'proporcion_de_energias_renovables_del_total_consumido':'prop_ren'})

# TS1 (temperatura)

In [19]:
ts1.tail()

,Anio,Temperatura
116,2017,24.280000
117,2018,23.961562
118,2019,24.290000
119,2020,24.404688
120,2021,24.156875


In [347]:
df=ts1.rename(columns={'Anio':'ds','Temperatura':'y'})
df['ds'] = pd.to_datetime(df.ds,format='%Y')

In [205]:
#separo en train y test
end_date = '1999-12-31'
df_train = (df[df['ds'] <= end_date])
df_test = (df[df['ds'] > end_date])

In [294]:
from sklearn.model_selection import train_test_split as tts
df_train, df_test= tts(df, test_size=0.2, random_state=0)

In [299]:
df_train.sort_values(by='ds')

,ds,y
0,2000-01-01,29.525806
2,2002-01-01,29.187419
3,2003-01-01,28.799677
4,2004-01-01,28.597097
5,2005-01-01,28.419032
6,2006-01-01,27.507419
7,2007-01-01,26.905161
9,2009-01-01,26.735161
10,2010-01-01,26.469677
11,2011-01-01,26.510645


In [349]:
regresion =Prophet(changepoint_prior_scale = 0.3,
                         n_changepoints = 200,
                         seasonality_mode = 'additive',
                         weekly_seasonality=True,
                         daily_seasonality = True,
                         yearly_seasonality = True, 
                         interval_width=0.95)
regresion.fit(df)

05:31:16 - cmdstanpy - INFO - Chain [1] start processing
05:31:17 - cmdstanpy - INFO - Chain [1] done processing


In [350]:
tuned_forecast = regresion.make_future_dataframe(periods=9, freq='Y')


In [70]:
tuned_forecast[tuned_forecast.ds < '1999-12-31']

,ds
0,1901-01-01
1,1902-01-01
2,1903-01-01
3,1904-01-01
4,1905-01-01
...,...
94,1995-01-01
95,1996-01-01
96,1997-01-01
97,1998-01-01


In [351]:
forecast = regresion.predict(tuned_forecast)
plot_plotly(regresion,forecast)

In [73]:
RMSE = mean_squared_error(df_train['y'], forecast[forecast.ds<'1999-12-31'].yhat,squared=False)

RMSE

0.13175279182901348

In [78]:
RMSE = mean_squared_error(df_test['y'], forecast[forecast.ds>='1999-12-31'].yhat,squared=False)

RMSE

0.1637424077406141

In [224]:
ts1_2030=forecast.tail(1)[['yhat', 'yhat_lower', 'yhat_upper']].reset_index(drop=True)

,yhat,yhat_lower,yhat_upper
0,24.286279,24.002368,24.585898


In [76]:
from sklearn.metrics import r2_score

In [81]:
r2_score(df_train['y'], forecast[forecast.ds<'1999-12-31'].yhat)

0.7525207078839264

In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df['ds'], y = df['y'], name = 'Temperatura'))
fig.add_trace(go.Scatter(x = df['ds'], y = df['y'], mode = 'markers', name = 'Valores'))
fig.show()

In [258]:
ts1_2030=forecast.tail(1)[['yhat', 'yhat_lower', 'yhat_upper']].reset_index(drop=True)

In [259]:
ts1_2030

,yhat,yhat_lower,yhat_upper
0,24.286279,23.999038,24.581797


# ts2 (Emisiones_de_CO2)

In [82]:
ts2.head()

,Anio,Emisiones_de_CO2
0,1980,4946.627130
1,1981,5930.700732
2,1982,5791.340244
3,1983,5544.802438
4,1984,5495.316484


In [225]:
df=ts2.rename(columns={'Anio':'ds','Emisiones_de_CO2':'y'})
df['ds'] = pd.to_datetime(df.ds,format='%Y')

In [192]:
#separo en train y test
end_date = '2009-12-31'
df_train = (df[df['ds'] <= end_date])
df_test = (df[df['ds'] > end_date])

In [226]:
regresion =Prophet(changepoint_prior_scale = 0.3,
                         n_changepoints = 200,
                         seasonality_mode = 'additive',
                         weekly_seasonality=True,
                         daily_seasonality = True,
                         yearly_seasonality = True, 
                         interval_width=0.95)
regresion.fit(df)

17:52:48 - cmdstanpy - INFO - Chain [1] start processing
17:52:49 - cmdstanpy - INFO - Chain [1] done processing


In [227]:
tuned_forecast = regresion.make_future_dataframe(periods=11, freq='Y')

In [228]:
forecast = regresion.predict(tuned_forecast)
plot_plotly(regresion,forecast)

In [204]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df['ds'], y = df['y'], name = 'Temperatura'))
fig.add_trace(go.Scatter(x = df['ds'], y = df['y'], mode = 'markers', name = 'Valores'))
fig.show()

In [196]:
RMSE = mean_squared_error(df['y'], forecast[forecast.ds<'2019-12-31'].yhat,squared=False)

RMSE

30.760677348779247

In [197]:
r2_score(df['y'], forecast[forecast.ds<'2019-12-31'].yhat)

0.9918513033337895

In [229]:
ts2_2030=forecast.tail(1)[['yhat', 'yhat_lower', 'yhat_upper']].reset_index(drop=True)

# ts3 (intensidad)

In [101]:
ts3.tail()

,Anio,intensidad
15,2015,4.048065
16,2016,4.042903
17,2017,3.970323
18,2018,3.983226
19,2019,4.032581


In [306]:
df=ts3.rename(columns={'Anio':'ds','intensidad':'y'})
df['ds'] = pd.to_datetime(df.ds,format='%Y')

In [99]:
end_date = '2014-12-31'
df_train = (df[df['ds'] <= end_date])
df_test = (df[df['ds'] > end_date])

In [307]:
regresion = Prophet()
regresion.fit(df)

05:05:14 - cmdstanpy - INFO - Chain [1] start processing
05:05:15 - cmdstanpy - INFO - Chain [1] done processing


In [308]:
future = regresion.make_future_dataframe(periods=11)
anio = 2000
for i in range(20,51):
  future.loc[i] = anio
  anio+=1
for i in range(20,51):
  future.loc[i] = pd.to_datetime(future.loc[i],format='%Y')

In [309]:
anio2000 = 2000
anio2000 = pd.to_datetime(anio2000,format='%Y')

In [310]:
forecast=regresion.predict(future)
plot_plotly(regresion,forecast[forecast.ds >= anio2000])

In [314]:
start_date = '1999-12-31'
end_date = '2019-12-31'

In [315]:
RMSE = mean_squared_error(df['y'],abs(forecast[['ds', 'yhat']].drop_duplicates()[(forecast.ds>start_date) & (forecast.ds<end_date)]['yhat']),squared=False)

RMSE

C:\Users\Gerardo\AppData\Local\Temp\ipykernel_11712\3786172639.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.045457248566978865

In [317]:
r2_score(df['y'], forecast[['ds', 'yhat']].drop_duplicates()[(forecast.ds>start_date) & (forecast.ds<end_date)]['yhat'])

C:\Users\Gerardo\AppData\Local\Temp\ipykernel_11712\1611156668.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.9212003008155876

In [318]:
ts3_2030=forecast.tail(1)[['yhat', 'yhat_lower', 'yhat_upper']].reset_index(drop=True)

# ts4 (prom)

In [106]:
ts4.head()

,Anio,prom
0,2000,80.739456
1,2001,81.370145
2,2002,81.984427
3,2003,82.458258
4,2004,83.007671


In [325]:
df=ts4.rename(columns={'Anio':'ds','prom':'y'})
df['ds'] = pd.to_datetime(df.ds,format='%Y')

In [ ]:
end_date = '2014-12-31'
df_train = (df[df['ds'] <= end_date])
df_test = (df[df['ds'] > end_date])

In [326]:
regresion =Prophet(changepoint_prior_scale = 0.3,
                         n_changepoints = 200,
                         seasonality_mode = 'additive',
                         weekly_seasonality=True,
                         daily_seasonality = True,
                         yearly_seasonality = True, 
                         interval_width=0.95)
regresion.fit(df)

05:08:29 - cmdstanpy - INFO - Chain [1] start processing
05:08:56 - cmdstanpy - INFO - Chain [1] done processing


In [327]:
tuned_forecast = regresion.make_future_dataframe(periods=11, freq='Y')

In [328]:
forecast = regresion.predict(tuned_forecast)
plot_plotly(regresion,forecast)

In [331]:
RMSE = mean_squared_error(df['y'],abs(forecast[['ds', 'yhat']].drop_duplicates()[(forecast.ds>start_date) & (forecast.ds<end_date)]['yhat']),squared=False)

RMSE

TypeError: loop of ufunc does not support argument 0 of type float which has no callable sqrt method

In [329]:
r2_score(df['y'], forecast[forecast.ds<'2019-12-31'].yhat)

0.9999976025826612

In [332]:
ts4_2030=forecast.tail(1)[['yhat', 'yhat_lower', 'yhat_upper']].reset_index(drop=True)

Pese a los valores, es un modelo overfitteado y se ve claramente en la incertidumbre diveregente de la predicción

# ts5 (prop_ren)

In [132]:
ts5.tail()

,Anio,prop_ren
15,2015,24.728750
16,2016,24.733125
17,2017,25.166563
18,2018,25.095938
19,2019,24.137500


In [333]:
df=ts5.rename(columns={'Anio':'ds','prop_ren':'y'})
df['ds'] = pd.to_datetime(df.ds,format='%Y')

In [243]:
end_date = '2014-12-31'
df_train = (df[df['ds'] <= end_date])
df_test = (df[df['ds'] > end_date])

In [334]:
regresion = Prophet()
regresion.fit(df)

05:09:57 - cmdstanpy - INFO - Chain [1] start processing
05:09:58 - cmdstanpy - INFO - Chain [1] done processing


In [335]:
future = regresion.make_future_dataframe(periods=11)
anio = 2000
for i in range(20,51):
  future.loc[i] = anio
  anio+=1
for i in range(20,51):
  future.loc[i] = pd.to_datetime(future.loc[i],format='%Y')

In [336]:
anio2000 = 2000
anio2000 = pd.to_datetime(anio2000,format='%Y')
forecast=regresion.predict(future)
plot_plotly(regresion,forecast[forecast.ds >= anio2000])

In [337]:
start_date = '1999-12-31'
end_date = '2019-12-31'

In [338]:
RMSE = mean_squared_error(df['y'],abs(forecast[['ds', 'yhat']].drop_duplicates()[(forecast.ds>start_date) & (forecast.ds<end_date)]['yhat']),squared=False)

RMSE

C:\Users\Gerardo\AppData\Local\Temp\ipykernel_11712\3786172639.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.30577317994040143

In [339]:
r2_score(df['y'], forecast[['ds', 'yhat']].drop_duplicates()[forecast.ds<'2019-12-31'].yhat)

C:\Users\Gerardo\AppData\Local\Temp\ipykernel_11712\1276589135.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.9624556930148327

In [340]:
ts5_2030=forecast.tail(1)[['yhat', 'yhat_lower', 'yhat_upper']].reset_index(drop=True)

# Predicciones por cada dataset

In [341]:
predicciones=pd.concat([ts1_2030, ts2_2030, ts3_2030, ts4_2030, ts5_2030])

In [342]:
predicciones.reset_index(drop=True, inplace=True)

In [343]:
predicciones.columns = ['prediccion', 'inc_baja', 'inc_alta']

In [344]:
predicciones.insert(3, 'dataset', ['temperatura', 'emisiones_co2', 'intensidad_energetica', 'acceso', 'energia_renovable'], allow_duplicates=False)

In [345]:
predicciones

,prediccion,inc_baja,inc_alta,dataset
0,24.286279,23.999038,24.581797,temperatura
1,2108.717199,1941.235206,2277.754416,emisiones_co2
2,3.637970,3.572956,3.699884,intensidad_energetica
3,93.979211,89.605331,98.355667,acceso
4,21.761339,21.343719,22.161613,energia_renovable


In [346]:
predicciones.to_csv('predicciones.csv')